In [1]:
# Dependencies
import json
import requests
import pprint
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#Import data as pandas DF's
#Education_2016_df = pd.read_csv("Resources/2016_Education_Postcode.csv")
#Income_2016_df = pd.read_csv("Resources/2016_Income_Postcode.csv")
#VehicleCount_2016_df = pd.read_csv("Resources/2016_VehicleCount_Postcode.csv")
Education_2021_df = pd.read_csv("Resources/2021_Education_Postcode.csv")
Income_2021_df = pd.read_csv("Resources/2021_Income_Postcode.csv")
VehicleCount_2021_df = pd.read_csv("Resources/2021_VehicleCount_Postcode.csv")

#adjust name once year confirmed
FuelType_2021_df = pd.read_csv("Resources/FuelType_Postcode.csv")
#pd.read_csv("Resources/2016_FuelType_Postcode.csv")

In [3]:
def create_scatter(df, y_variable, x_variable):

    x_values = df[x_variable]
    y_values = df[y_variable]
    
    #get plot axis range for annotation location.
    ymin = df[y_variable].min()
    xmin = df[x_variable].min()

    ymax = df[y_variable].max()
    xmax = df[x_variable].max()

    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

    plt.scatter(
        x_values,
        y_values,
        color = 'blue',
        marker = 'o',
        s = 50,
        edgecolors = 'black')

    plt.plot(x_values, regress_values, "r-", linewidth = 1)

    #plt.annotate(line_eq, (xmin + 1, ymin + 5), fontsize =10, color = "red")

    plt.xlabel('Latitude')
    plt.ylabel(y_variable)
    plt.suptitle(f"The r-value is: {rvalue}", fontsize = 8, x = 0.2, y = 0.95)
    plt.annotate(line_eq, xy=(0.8, 1.05), fontsize =8, color = "red",
            xycoords='axes fraction', textcoords='axes fraction')

    plt.show()


In [4]:
def reshape_df(df):
    return df.pivot_table(index = ['Vehicle_type', 'Postcode'], 
                          columns = 'Fuel_type', 
                          values = 'Count').reset_index().rename_axis(None, axis=1)

In [5]:
def clean_headers(df):
    df.columns = df.columns.str.replace(" ", "_")
    return df

Clean Data

In [6]:
#Clean Headers
#Education_2016_df = clean_headers(Education_2016_df)
#Income_2016_df = clean_headers(Income_2016_df)
#VehicleCount_2016_df = clean_headers(VehicleCount_2016_df)
Education_2021_df = clean_headers(Education_2021_df)
Income_2021_df = clean_headers(Income_2021_df)
VehicleCount_2021_df = clean_headers(VehicleCount_2021_df)
FuelType_2021_df = clean_headers(FuelType_2021_df)

In [7]:
FuelType_2021_df = reshape_df(FuelType_2021_df)
#FuelType_2016_df = reshape_df(FuelType_2016_df)


In [8]:
FuelType_2021_df = FuelType_2021_df[FuelType_2021_df["Vehicle_type"] != "Total"]
FuelType_2021_df = FuelType_2021_df[FuelType_2021_df["Postcode"] != "Total"]


Merge DFs so that Education, income, vehicle count, fuel type are in 1 df on postcode handle different postcodes that arent present in all datasets

In [9]:
#Combine the 2016 data into a single dataset.
#data_education_income_2016 = pd.merge(Education_2016_df, Income_2016_df, how="inner", on= ["Postcode", "Postcode"])
#data_complete_2016 = pd.merge(data_education_income_2016, VehicleCount_2016_df ,how="inner", on= ["Postcode", "Postcode"])
#data_complete_2016 = data_complete_2016.drop(['State','Total_y', 'Total'], axis=1)

#data_complete_2016.head()

,Postcode,State_x,Postgraduate_Degree_Level,Graduate_Diploma_and_Graduate_Certificate_Level,Bachelor_Degree_Level,Advanced_Diploma_and_Diploma_Level,Certificate_III_&_IV_Level,Secondary_Education_-_Years_10_and_above,Certificate_I_&_II_Level,Secondary_Education_-_Years_9_and_below,...,"$3,000_or_more_($156,000_or_more)",Not_stated_y,Not_applicable_y,No_motor_vehicles,One_motor_vehicle,Two_motor_vehicles,Three_motor_vehicles,Four_or_more_motor_vehicles,Not_stated,Not_applicable
0,2000,NSW,2836,307,7278,2852,931,6241,9,443,...,1796,4115,1251,5196,3229,672,63,41,2163,2473
1,2006,NSW,34,4,114,12,10,912,0,5,...,11,156,15,0,4,0,0,0,3,6
2,2007,NSW,903,95,2537,622,275,2496,0,200,...,196,1072,375,1656,974,144,16,20,607,247
3,2008,NSW,1387,174,3549,727,389,3269,0,142,...,359,1359,407,2738,1402,298,34,21,704,428
4,2009,NSW,1583,281,3653,1099,659,2387,3,244,...,1271,1254,1187,1617,2793,792,83,26,629,527


In [10]:
Education_2021_df['Postcode'] = Education_2021_df['Postcode'].astype(int)
data_education_income_2021 = pd.merge(Education_2021_df, Income_2021_df, how="inner", on= ["Postcode", "Postcode"])
data_complete_2021 = pd.merge(data_education_income_2021, VehicleCount_2021_df ,how="inner", on= ["Postcode", "Postcode"])
data_complete_2021 = data_complete_2021.drop(['State','Total_y', 'Total'], axis=1)

Remove data for "Not Stated or Not Applicable" and subtract from Total column

In [13]:
#Education_2016_df["Total"] = Education_2016_df["Total"] - (Education_2016_df["Not_stated"] + Education_2016_df["Not_applicable"])
#Education_2016_df = Education_2016_df.drop(columns=["Not_stated", "Not_applicable"])

Education_2021_df["Total"] = Education_2021_df["Total"] - (Education_2021_df["Not_stated"] + Education_2021_df["Not_applicable"])
Education_2021_df = Education_2021_df.drop(columns=["Not_stated", "Not_applicable"])


In [16]:
#Income_2016_df["Total"] = Income_2016_df["Total"] - (Income_2016_df["Not_stated"] + Income_2016_df["Not_applicable"])
#Income_2016_df = Income_2016_df.drop(columns=["Not_stated", "Not_applicable"])

Income_2021_df["Total"] = Income_2021_df["Total"] - (Income_2021_df["Not_stated"] + Income_2021_df["Not_applicable"])
Income_2021_df = Income_2021_df.drop(columns=["Not_stated", "Not_applicable"])

In [19]:
#Same for people with multiple cars?
Education_2021_df

,Postcode,State,Postgraduate_Degree_Level,Graduate_Diploma_and_Graduate_Certificate_Level,Bachelor_Degree_Level,Advanced_Diploma_and_Diploma_Level,Certificate_III_&_IV_Level,Secondary_Education_-_Years_10_and_above,Certificate_I_&_II_Level,Secondary_Education_-_Years_9_and_below,Supplementary_Codes,Total
0,2000,NSW,3813,489,8530,3814,1137,4761,16,397,872,23818
1,2007,NSW,1134,134,2165,722,347,1426,0,163,202,6285
2,2008,NSW,1731,228,3450,752,404,2265,0,97,147,9073
3,2009,NSW,1872,346,3673,1162,712,2007,12,223,329,10324
4,2010,NSW,4003,796,9068,2185,1402,4145,6,454,602,22655
...,...,...,...,...,...,...,...,...,...,...,...,...
2636,4377,QLD/NSW,9,3,31,35,120,184,0,52,7,441
2637,4380,QLD/NSW,155,90,680,509,1292,2293,10,754,213,5993
2638,4383,QLD/NSW,3,3,15,30,112,179,0,88,12,445
2639,4385,QLD/NSW,6,21,61,67,142,350,0,168,20,839


In [18]:
#Same for vehicle types, probably want to filter out
FuelType_2021_df

,Vehicle_type,Postcode,Diesel,Electric,LPG/Dual fuel,Not Applicable,Other and unknown,Petrol,Total
0,Light Commercial Vehicles,1225,3,0,0,0,0,0,3
1,Light Commercial Vehicles,1232,0,0,0,0,0,0,0
2,Light Commercial Vehicles,1235,0,0,0,0,0,0,0
3,Light Commercial Vehicles,1335,0,0,0,0,0,0,0
4,Light Commercial Vehicles,1340,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
5594,Passenger Vehicles,885,132,0,0,0,0,160,288
5595,Passenger Vehicles,886,187,0,0,0,0,218,401
5596,Passenger Vehicles,9000,0,0,0,0,0,3,3
5597,Passenger Vehicles,9726,0,0,0,0,0,0,0


Create DF's with normalised values (percentage of total)

In [25]:
Education_2021_Normalized = Education_2021_df.copy()
for col in Education_2021_Normalized.columns[2:-1]:
    Education_2021_Normalized[col] = Education_2021_Normalized[col] / Education_2021_Normalized["Total"] * 100
Education_2021_Normalized = Education_2021_Normalized.round(2)


In [27]:
FuelType_2021_Normalized = FuelType_2021_df.copy()
for col in FuelType_2021_Normalized.columns[2:-1]:
    FuelType_2021_Normalized[col] = FuelType_2021_Normalized[col] / FuelType_2021_Normalized["Total"] * 100
FuelType_2021_Normalized = FuelType_2021_Normalized.round(2)

#if total = 0 then drop row?
FuelType_2021_Normalized

,Vehicle_type,Postcode,Diesel,Electric,LPG/Dual fuel,Not Applicable,Other and unknown,Petrol,Total
0,Light Commercial Vehicles,1225,100.00,0.00,0.00,0.0,0.00,0.00,3
1,Light Commercial Vehicles,1232,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Light Commercial Vehicles,1235,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Light Commercial Vehicles,1335,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Light Commercial Vehicles,1340,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
5594,Passenger Vehicles,885,45.83,0.00,0.00,0.0,0.00,55.56,288
5595,Passenger Vehicles,886,46.63,0.00,0.00,0.0,0.00,54.36,401
5596,Passenger Vehicles,9000,0.00,0.00,0.00,0.0,0.00,100.00,3
5597,Passenger Vehicles,9726,NaN,NaN,NaN,NaN,NaN,NaN,0


In [31]:
Income_2021_Normalized = Income_2021_df.copy()
for col in Income_2021_Normalized.columns[2:-1]:
    Income_2021_Normalized[col] = Income_2021_Normalized[col] / Income_2021_Normalized["Total"] * 100
Income_2021_Normalized = Income_2021_Normalized.round(2)
Income_2021_Normalized

,Postcode,State,Negative_income,Nil_income,"$1-$149_($1-$7,799)","$150-$299_($7,800-$15,599)","$300-$399_($15,600-$20,799)","$400-$499_($20,800-$25,999)","$500-$649_($26,000-$33,799)","$650-$799_($33,800-$41,599)","$800-$999_($41,600-$51,999)","$1,000-$1,249_($52,000-$64,999)","$1,250-$1,499_($65,000-$77,999)","$1,500-$1,749_($78,000-$90,999)","$1,750-$1,999_($91,000-$103,999)","$2,000-$2,999_($104,000-$155,999)","$3,000-$3,499_($156,000-$181,999)","$3,500_or_more_($182,000_or_more)",Total
0,2000,NSW,0.66,9.80,1.35,2.42,4.32,6.31,8.79,9.13,10.13,9.92,6.45,5.19,4.28,9.59,3.09,8.54,24116
1,2007,NSW,0.86,12.37,1.84,3.79,6.16,7.15,10.63,8.90,8.55,9.97,6.76,4.89,3.96,8.24,2.61,3.46,6360
2,2008,NSW,0.74,15.54,2.82,3.53,5.28,5.23,7.49,7.21,7.35,8.51,7.26,6.09,5.18,10.35,2.66,4.80,9146
3,2009,NSW,0.39,7.51,1.48,2.63,4.04,4.24,5.67,6.23,7.35,9.02,7.75,7.09,6.11,13.84,4.78,11.92,10429
4,2010,NSW,0.33,3.95,1.14,2.24,4.32,4.87,5.09,5.55,6.78,8.54,7.83,7.89,6.64,16.17,5.27,13.41,22730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,4377,QLD/NSW,0.00,4.70,3.80,8.95,13.87,12.30,12.53,8.28,13.65,7.16,4.47,3.36,1.57,1.57,0.00,0.89,447
2637,4380,QLD/NSW,0.87,5.42,4.03,6.92,14.52,13.32,10.68,9.81,9.76,8.51,5.27,3.74,2.16,3.10,0.70,1.14,6124
2638,4383,QLD/NSW,0.00,9.53,3.39,6.99,19.70,17.80,9.53,6.14,7.84,8.05,4.03,1.91,1.48,1.27,0.85,1.27,472
2639,4385,QLD/NSW,2.07,4.83,3.22,8.39,19.31,15.86,9.66,5.17,7.36,8.51,4.60,4.14,1.95,3.33,0.34,1.38,870


Group into bigger bins for simplicity of plotting.
Make new dfs with created bins

Create Plots